In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.18.0-dev20240813


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-08-15 04:11:21--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2024-08-15 04:11:22 (7.13 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-08-15 04:11:22--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [ ]:
df_train = pd.read_csv(train_file_path, sep = '\t', header = None, names = ['Class', 'Message'])
df_test = pd.read_csv(test_file_path, sep = '\t', header = None, names = ['Class', 'Message'])

In [ ]:
print(df_train.shape)
print(df_test.shape)

(4179, 2)
(1392, 2)


In [ ]:
df_train.head(10)

,Class,Message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
5,ham,in xam hall boy asked girl tell me the startin...
6,ham,genius what's up. how your brother. pls send h...
7,ham,they finally came to fix the ceiling.
8,spam,urgent! call 09066350750 from your landline. y...
9,ham,now that you have started dont stop. just pray...


In [ ]:
df_test.head(10)

,Class,Message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
5,ham,your opinion about me? 1. over 2. jada 3. kusr...
6,ham,yesterday its with me only . now am going home.
7,ham,yes. it's all innocent fun. o:-)
8,ham,a boy was late 2 home. his father: power of fr...
9,ham,is ur changes 2 da report big? cos i've alread...


In [ ]:
print(df_train.isna().sum())
print(df_test.isna().sum())

Class      0
Message    0
dtype: int64
Class      0
Message    0
dtype: int64


In [ ]:
print(df_train.dtypes)
print(df_test.dtypes)

Class      object
Message    object
dtype: object
Class      object
Message    object
dtype: object


In [ ]:
# Create dummy variables

df_train = pd.get_dummies(df_train, columns = ['Class'], drop_first = True)
df_test = pd.get_dummies(df_test, columns = ['Class'], drop_first = True)
print(df_train.head())
print(df_test.head())

                                             Message  Class_spam
0  ahhhh...just woken up!had a bad dream about u ...       False
1                           you can never do nothing       False
2  now u sound like manky scouse boy steve,like! ...       False
3  mum say we wan to go then go... then she can s...       False
4  never y lei... i v lazy... got wat? dat day ü ...       False
                                             Message  Class_spam
0  i am in hospital da. . i will return home in e...       False
1         not much, just some textin'. how bout you?       False
2  i probably won't eat at all today. i think i'm...       False
3  don‘t give a flying monkeys wot they think and...       False
4                                who are you seeing?       False


In [ ]:
df_train['Class_spam'] = df_train['Class_spam'].astype(int)
df_test['Class_spam'] = df_test['Class_spam'].astype(int)
print(df_train.head())
print(df_test.head())

                                             Message  Class_spam
0  ahhhh...just woken up!had a bad dream about u ...           0
1                           you can never do nothing           0
2  now u sound like manky scouse boy steve,like! ...           0
3  mum say we wan to go then go... then she can s...           0
4  never y lei... i v lazy... got wat? dat day ü ...           0
                                             Message  Class_spam
0  i am in hospital da. . i will return home in e...           0
1         not much, just some textin'. how bout you?           0
2  i probably won't eat at all today. i think i'm...           0
3  don‘t give a flying monkeys wot they think and...           0
4                                who are you seeing?           0


In [ ]:
# Data processing

def text_processing(df):

  import string
  import re

  # Eliminate punctuation signs and number

  punctuation = string.punctuation
  spaces = ' ' * len(punctuation)
  df['Message'] = df['Message'].apply(lambda x: x.translate(str.maketrans(punctuation, spaces)))
  df['Message'] = df['Message'].apply(lambda x: re.sub(r'\d+', '', x))

  # Tokenize the messages

  df['Message'] = df['Message'].astype(str)
  df['Message'] = df['Message'].apply(lambda x: x.split())

  return df

In [ ]:
df_train = text_processing(df_train)
df_test = text_processing(df_test)

In [ ]:
df_train.head()

,Message,Class_spam
0,"[ahhhh, just, woken, up, had, a, bad, dream, a...",0
1,"[you, can, never, do, nothing]",0
2,"[now, u, sound, like, manky, scouse, boy, stev...",0
3,"[mum, say, we, wan, to, go, then, go, then, sh...",0
4,"[never, y, lei, i, v, lazy, got, wat, dat, day...",0


In [ ]:
df_test.head()

,Message,Class_spam
0,"[i, am, in, hospital, da, i, will, return, hom...",0
1,"[not, much, just, some, textin, how, bout, you]",0
2,"[i, probably, won, t, eat, at, all, today, i, ...",0
3,"[don‘t, give, a, flying, monkeys, wot, they, t...",0
4,"[who, are, you, seeing]",0


In [ ]:
# Create the vocabulary

vocabulary = [word for message in df_train['Message'] for word in message]
vocabulary = list(set(vocabulary))
total_words = len(vocabulary) + 1
print(total_words)

6782


In [ ]:
# Getting the max length of a message

max_length = max(len(message) for message in df_train['Message'])
print(max_length)

190


In [ ]:
# Create an encoder dictionary

encoder_dict = {}

for word in vocabulary:
    if word not in encoder_dict:
        encoder_dict[word] = len(encoder_dict) + 1

print(encoder_dict)

{'inform': 1, 'latests': 2, 'weakness': 3, 'suman': 4, 'curfew': 5, 'hour': 6, 'coat': 7, 'lady': 8, 'tx': 9, 'chapel': 10, 'smiled': 11, 'instant': 12, 'learn': 13, 'logo': 14, 'scallies': 15, 'txtstop': 16, 'dificult': 17, 'sudn': 18, 'describe': 19, 'computational': 20, 'burning': 21, 'province': 22, 'recovery': 23, 'jokes': 24, 'dare': 25, 'presnts': 26, 'ended': 27, 'letter': 28, 'big': 29, 'cha': 30, 'impressed': 31, 'use': 32, 'maths': 33, 'howu': 34, 'matter': 35, 'argentina': 36, 'radio': 37, 'christians': 38, 'evrey': 39, 'instead': 40, 'thew': 41, 'pura': 42, 'bluetooth': 43, 'forth': 44, 'loosing': 45, 'bill': 46, 'bowl': 47, 'got': 48, 'kaiez': 49, 'hg': 50, 'ha': 51, 'pay': 52, 'posted': 53, 'virtual': 54, 'maybe': 55, 'avble': 56, 'webeburnin': 57, 'moby': 58, 'copied': 59, 'wrote': 60, 'pattern': 61, 'dreamz': 62, 'wanna': 63, 'finest': 64, 'cme': 65, 'bite': 66, 'rael': 67, 'capacity': 68, 'blessings': 69, 'enc': 70, 'lie': 71, 'alaipayuthe': 72, 'visionsms': 73, 'corr

In [ ]:
def encode(df, encoder):
  df['Message'] = df['Message'].apply(lambda x: [encoder.get(word, -1) for word in x])
  return df

In [ ]:
encode(df_train, encoder_dict)
encode(df_test, encoder_dict)
print(df_train.head())
print(df_test.head())

                                             Message  Class_spam
0  [350, 730, 2642, 6019, 2355, 3033, 1590, 4776,...           0
1                     [3973, 2864, 3199, 6259, 1576]           0
2  [5007, 6421, 5105, 873, 1159, 4046, 2665, 5184...           0
3  [1145, 1713, 5248, 3731, 2722, 4476, 4891, 447...           0
4  [3199, 944, 5650, 6588, 6658, 6553, 48, 6583, ...           0
                                             Message  Class_spam
0  [6588, 215, 4151, 2096, 4200, 6588, 6037, 6162...           0
1      [4323, 6591, 730, 2617, -1, 1609, 3383, 3973]           0
2  [6588, 1162, 404, 3740, 601, 589, 5112, 4811, ...           0
3  [5508, 5244, 3033, -1, -1, 4924, 128, 3905, 50...           0
4                           [2123, 3747, 3973, 1685]           0


In [ ]:
# Create train and test labels and convert them to numpy arrays

train_labels = df_train.pop('Class_spam').to_numpy()
test_labels = df_test.pop('Class_spam').to_numpy()


df_train = np.squeeze(df_train.to_numpy())
df_test = np.squeeze(df_test.to_numpy())

print(df_train)
print(df_test)
print(train_labels)
print(test_labels)

[list([350, 730, 2642, 6019, 2355, 3033, 1590, 4776, 4294, 6421, 6441, 2948, 6588, 5372, 873, 6421, 6494, 5007, 6588, 1390, 6431, 5974, 4294, 4094, 6024, 505, 6588, 6645, 5308, 6019, 3600, 3349])
 list([3973, 2864, 3199, 6259, 1576])
 list([5007, 6421, 5105, 873, 1159, 4046, 2665, 5184, 873, 6588, 6404, 2156, 6257, 4200, 6014, 1141, 4924, 4983, 6421, 1408, 4663, 2275, 6680])
 ...
 list([6760, 4722, 3994, 651, 1418, 4268, 2514, 730, 6535, 4025, 5995, 5013, 2722, 5007, 3740, 1332, 3954, 752, 1880, 5663])
 list([2673, 5355, 2539, 4772, 6404, 2202, 3973, 2722, 1629, 1362, 4452, 4407, 4130, 4841, 2328, 4300, 1362, 3954, 2129, 1286, 6421, 2539, 5355, 5264, 5355, 3429, 2722])
 list([4919, 5744, 1590, 111, 2135, 6421, 1870, 605, 4151, 6270, 6588, 6215, 3432, 2722, 469, 4151, 2528, 2143, 6588, 6215, 6734, 2585, 5744, 3527, 2463, 709, 851, 3600, 5266, 4407, 1629, 4982, 2281, 3616])]
[list([6588, 215, 4151, 2096, 4200, 6588, 6037, 6162, 1141, 4151, 5250])
 list([4323, 6591, 730, 2617, -1, 1609, 3

In [ ]:
print(df_train[0])

[350, 730, 2642, 6019, 2355, 3033, 1590, 4776, 4294, 6421, 6441, 2948, 6588, 5372, 873, 6421, 6494, 5007, 6588, 1390, 6431, 5974, 4294, 4094, 6024, 505, 6588, 6645, 5308, 6019, 3600, 3349]


In [ ]:
# Data homogeneity

data_length = max_length // 2
from keras.preprocessing import sequence

df_train = sequence.pad_sequences(df_train, data_length)
df_test = sequence.pad_sequences(df_test, data_length)

In [ ]:
print(df_train[0])
print(df_test[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0  350  730 2642 6019 2355 3033 1590
 4776 4294 6421 6441 2948 6588 5372  873 6421 6494 5007 6588 1390 6431
 5974 4294 4094 6024  505 6588 6645 5308 6019 3600 3349]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
 6588  215 4151 2096 4200 6588 6037 6162 1141 4151 5250]


In [ ]:
# Building the model

model = keras.Sequential([
    keras.layers.Embedding(total_words, 62),
    keras.layers.LSTM(62),
    keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Training the model

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
history = model.fit(df_train, train_labels, epochs = 10)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - acc: 0.8794 - loss: 0.3292
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - acc: 0.9869 - loss: 0.0484
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - acc: 0.9949 - loss: 0.0236
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - acc: 0.9986 - loss: 0.0067
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - acc: 0.9986 - loss: 0.0059
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - acc: 0.9997 - loss: 0.0025
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - acc: 0.9998 - loss: 0.0016
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - acc: 0.9986 - loss: 0.0026
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - acc: 0.9998 - loss: 6.8696e-04
Epoch 10/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - acc: 1.0000 - loss: 7.2180e-04


In [ ]:
results = model.evaluate(df_test, test_labels)
print(results)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - acc: 0.9865 - loss: 0.0810
[0.07091935724020004, 0.9877873659133911]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):

  if isinstance(pred_text, str):
    pred_text = [pred_text]

  predictions_list = []
  for text in pred_text:
    prediction = []

    # Data preprocessing

    data_dict = {'Message': [text]}
    data = pd.DataFrame(data_dict)

    data = text_processing(data)
    data = encode(data, encoder_dict)
    data = np.squeeze(data.to_numpy())
    data = np.expand_dims(data, axis = 0)
    data = sequence.pad_sequences(data, data_length)

    pred = model.predict(data)
    prediction.append(pred[0][0])

    if pred[0][0] <= 0.5:
      label = "ham"
      prediction.append(label)
    else:
      label = "spam"
      prediction.append(label)

    predictions_list.append(prediction)

  predictions = []

  for list_value in predictions_list:
    for element in list_value:
      predictions.append(element)

  return predictions

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[3.1821848e-05, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
You passed the challenge. Great job!
